# 本章简介
本章，我们将通过对算法进行调优来构建性能良好的机器学习模型，并对模型性能进行评估：
+ 模型性能的无偏估计
+ 处理机器学习中的常见问题
+ 机器学习模型调优
+ 使用不同的性能指标评估预测模型 5

# 6.1 基于流水线的工作流
> 介绍了如何使用sklearn中的工具类pipline使得工作更加高效  

sklearn中的pipline类使得我们可以拟合出包含任意多个处理步骤的模型，并将模型用于新数据的预测 5  
## 6.1.1 加载威斯康星乳腺癌数据集
> 介绍了如何加载威斯康星乳腺癌数据集并进行预处理  

第一步：加载数据集

In [ ]:
'''
python	sklearn	Datasets	load_breast_cancer()
python	sklearn	Datasets	load_*() as_frame
python	sklearn	Datasets	load_*() frame
'''
from sklearn.datasets import load_breast_cancer
bc = load_breast_cancer(as_frame=True)
df = bc.frame
df.head()

第二步：对类标编码，此时恶性肿瘤和良性肿瘤分别被标识为类1和类0  

In [ ]:
'''
5
'''
import numpy as np
X = df.iloc[:, :-2].values
y = df.iloc[:, -1].values
np.unique(y)

第三步：我们将数据集划分为训练数据集(80%)和单独的测试数据集(20%)  

In [ ]:
'''
5
python	sklearn	Model Selection	train_test_split()
'''
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.20, random_state=1)

## 6.1.2 在流水线中集成数据转换及评估操作
> 介绍了如何通过pipeline实现预处理、训练等一系列操作以及pipeline的工作方式  

我们需要对数据特征列做标准化处理，此外，我们还想通过第5章介绍过的主成分分析降维到二维子空间，最后我们使用逻辑斯蒂回归模型分析数据  

In [ ]:
'''
python	sklearn	Preprocessing and Normalization	StandardScaler()
python	sklearn	Matrix Decomposition	PCA()
python	sklearn	pipeline	Pipeline()
python	sklearn	Pipeline	*.fit()
python	sklearn	Pipeline	*.score()
'''
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
pipe_lr = Pipeline([('scl', StandardScaler()),
                   ('pca', PCA(n_components=2)),
                   ('clf', LogisticRegression(random_state=1))])
pipe_lr.fit(X_train, y_train)
print('Test Accuracy: {:.3f}'.format(pipe_lr.score(X_test, y_test)))

Pipeline对象的输入可以是元组的序列，每个元组作为流水线的一个步骤，元组的第一个值必须是字符串，作为步骤的标识，可以用来访问具体的步骤元素，第二个值则为sklearn中的转换器、评估器  
当流水线上执行fit时，每个步骤会依次执行fit和transform操作，经过转换的数据会传递给流水线下一个对象 5   
流水线的工作方式可用下图来描述  
![6-1](../syn_pic/py_machine_learning/6-1.png)
5  
# 6.2 使用k折交叉验证评估模型性能 
> 介绍了用于模型性能评估的两种技术：holdout交叉验证和k折交叉验证的实现原理和方法  

如果一个模型过于简单会面临欠拟合（高偏差）问题，而模型过于复杂则会导致过拟合（高方差）问题。为了在偏差和方差之间找到折中的方案，我们需要对模型进行评估 5  
## 6.2.1 holdout方法
> 介绍了为何使用holdout交叉验证和其实现原理

为进一步提高模型在预测未知数据上的性能，我们还要进行模型选择过程，所谓模型选择过程：就是指针对给定分类问题我们调整参数并比较以寻求最优值  
在模型选择过程中，会产生一个问题，如果我们不断重复使用相同的测试数据，它们会成为训练数据的一部分，导致模型更易于过拟合  
所以更好的方法将数据划分为三个部分：训练数据集、验证数据集和测试数据集。其中训练数据集用于不同模型的拟合，验证数据集用于模型选择的标准  
下图介绍holdout交叉验证的概念 5  
![6-2](../syn_pic/py_machine_learning/6-2.png)
holdout方法的一个缺点在于：模型性能的评估对训练数据集划分为训练及验证子集的方法是敏感的，评价的结果会随样本的不同而发生变化 5  
## 6.2.2 k折交叉验证
> 介绍了k折交叉验证、分层k折交叉验证技术的实现原理和使用方法

k折交叉验证克服了holdout方法的缺点，对数据划分方法的敏感性较低，具有更好的鲁棒性  
其方法是不重复地随机将训练数据集划分为k个，其中k-1个用于模型的而训练，剩余的1个用于测试。随后重复此过程k次，我们就得到k个模型及对模型性能的评价  
基于这些独立且不同的数据子集上得到的模型性能评价结果，我们可以计算出其平均性能  
一旦找到了满意的超参值，我们就可以在全部训练集上重新训练模型，并使用独立测试集对模型做出最终评价 5  
由于k折交叉验证使用了无重复抽样，使得每个样本点只有一次被划入训练集或测试集的机会，与holdout方法相比，这将使得模型性能的评估具有较小的方差  
### 问题：为何无重复抽样技术可以使得模型性能评估具有较小方差?
> 待定  

下图总结了k折交叉验证的相关概念，其中k=10。$E_i$是第i次迭代的性能评价指标  
![6-3](../syn_pic/py_machine_learning/6-3.png)
k折交叉验证中的k标准值为10，但如果训练数据集相对较小，有必要加大k的值，这样会有更多的数据用于模型的训练，这样利用评估结果平均值对模型泛化性能进行评价时可以得到更小的偏差  
但是由于k值得增加导致交叉验证算法运行时间延长，而且各训练块间高度相似，也会导致评价结果方差较高。所以数据集较大，则可以选择较小得k值，如k=5 5   
k折交叉验证得一个特例就是留一(LOO)交叉验证法。在LOO中，我们将数据子集划分的数量等同于样本数(k=n)，这样每次只有一个样本用于测试，主要用于数据集非常小时的验证  
分层k折交叉验证对标准k折交叉验证做了稍许改进，它可以获得偏差和方差都较低的评估结果，特别时类别比例相差较大时  
在分k折交叉验证中，类别比例在每个分块中得以保持，下面通过sklearn中的stratifiedKFold迭代器来演示  

In [ ]:
'''
python	sklearn	Model Selection	StratifiedKFold()
python	sklearn	Model Selection	*.split()
python	numpy	Statistics	bincount()
'''
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=10)
scores = []
for k, (train, test) in enumerate(kfold.split(X_train, y_train)):
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr.score(X_train[test], y_train[test])
    scores.append(score)
    print('Fold: {}, Class dist.：{}， Acc：{:.3f}'.format(k+1, 
                                                        np.bincount(y_train[train]), score))

In [ ]:
'''
5
python	numpy	Statistics	mean()
python	numpy	Statistics	std()
'''
print('CV accuracy: {:.3f} +/- {:.3f}'.format(np.mean(scores),
                                             np.std(scores)))

我们test索引计算模型的准确率，将其存储在score列表中，用于计算平均准确率及性能评估标准差  
另外sklearn也同样实现了k折交叉验证评分的计算  

In [ ]:
'''
python	sklearn	Model Selection	cross_val_score()
'''
from sklearn.model_selection import cross_val_score
scores = cross_val_score(estimator=pipe_lr,
                        X=X_train,
                        y=y_train,
                        cv=10,
                        n_jobs=1)
print('CV accuracy scores:{}'.format(scores))

In [ ]:
print('CV accuracy: {:.3f} +/- {:.3f}'.format(np.mean(scores),
                                             np.std(scores)))

cross_val_score通过设定n_jobs可以支持多CPU并行运算 5  
# 6.3 通过学习及验证曲线来调试算法
> 本章介绍了如何使用两个有助于提高学习算法性能的判定工具：学习曲线与验证曲线 

学习曲线和验证曲线可以用来判定学习算法是否面临过拟合（高方差）或欠拟合（高偏差）问题 5  
## 6.3.1 使用学习曲线判定偏差和方差问题  
> 介绍了如何运用学习曲线来判定模型是否拟合良好，以及是否应该收集更多数据  

通过将模型的训练及准确性验证看作是训练数据集大小的函数，并绘制其图像，我们可以很容易看出模型是面临高方差还是高偏差的问题  
![6-4](../syn_pic/py_machine_learning/6-4.png)
左上方显示的是一个高偏差模型，解决此问题的常用方法是增加模型中参数的数量，例如，收集或构建额外特征，或者降低类似于SVM等模型的正则化程度  
右上方图像中的模型面临高方差的问题，表明训练准确度与交叉验证准确度之间有很大差距 5  
针对此类过拟合问题，我们可以收集更多的训练数据或者降低模型的复杂度，如正则化参数。或用特征抽取来降低特征的数量  
需要注意：收集更多数据不适用于所有问题，例如训练数据中噪声过多，或者模型本身接近最优  

In [ ]:
'''
5
python	sklearn	Pipeline	Pipeline()
python	sklearn	Model Selection	learning_curve()
python	matplotlib	Pyplot function overview	fill_between()
python	matplotlib	Pyplot function overview	grid()
b
'''
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
pipe_lr = Pipeline([
    ('scl', StandardScaler()),
    ('clf', LogisticRegression(
        penalty='l2', random_state=0))])
train_sizes, train_scores, test_scores =\
    learning_curve(estimator=pipe_lr,
                  X=X_train,
                  y=y_train,
                  train_sizes=np.linspace(0.1, 1.0, 10),
                  cv=10,
                  n_jobs=1)
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)
plt.plot(train_sizes, train_mean,
        color='blue', marker='o',
        markersize=5,
        label='training accuracy')
plt.fill_between(train_sizes,
                train_mean + train_std,
                train_mean - train_std,
                alpha=0.15, color='blue')
plt.plot(train_sizes, test_mean,
        color='green', linestyle='--',
        marker='s',markersize=5,
        label='validation accuracy')
plt.fill_between(train_sizes,
                test_mean + test_std,
                test_mean - test_std,
                alpha=0.15, color='green')
plt.grid()
plt.xlabel('Number of training samples')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0.8, 1.0])
plt.show()

通过learning_curve函数的train_sizes参数，我们可以控制用于生成学习曲线的样本的决定或相对数量。learning_curve函数使用分层K折交叉验证来计算交叉验证准确率  
我们通过fill_between函数加入了平均准确率标准差的信息，用以标识评价结果的标准差  
由于训练准确率曲线稍高于验证准确率，意味着模型对训练数据有轻微的过拟合，可以通过增大正则化来解决 5  
## 6.3.2 通过验证曲线来判定过拟合与欠拟合  
> 介绍了如何运用验证曲线来判定模型是否拟合良好，以及是否需要调整模型参数  

验证曲线与学习曲线相似，不过绘制的不是样本大小与训练准确率、测试准确率之间的函数关系，而是准确率与模型参数之间的关系  

In [ ]:
'''
python	sklearn	Model Selection	validation_curve()
'''
from sklearn.model_selection import validation_curve
param_range = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
train_scores, test_scores =\
    validation_curve(estimator=pipe_lr,
                  X=X_train,
                  y=y_train,
                  param_name='clf__C',
                  param_range=param_range,
                  cv=10)
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)
plt.plot(param_range, train_mean,
        color='blue', marker='o',
        markersize=5,
        label='training accuracy')
plt.fill_between(param_range,
                train_mean + train_std,
                train_mean - train_std,
                alpha=0.15, color='blue')
plt.plot(param_range, test_mean,
        color='green', linestyle='--',
        marker='s',markersize=5,
        label='validation accuracy')
plt.fill_between(param_range,
                test_mean + test_std,
                test_mean - test_std,
                alpha=0.15, color='green')
plt.grid()
plt.xscale('log')
plt.xlabel('Parameter C')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0.8, 1.0])
plt.show()

在validation_curve函数内，我们可以指定想要验证的参数。例如，需要验证参数C，即定义在流水线中逻辑斯蒂回归分类器的正则化参数，我们将其记为'clf_C'  
我们可以看到，如果加大正则化强度（较小的C值），会导致模型轻微的欠拟合;如果增加C的值，这意味着降低正则化强度，模型会趋向于过拟合，最优点在C=0.1附近 5  
# 6.4 使用网格搜索调优机器学习模型
> 介绍了调优模型超参的技术-网格搜索，以及在不同算法中做出选择的技术-嵌套交叉验证的概念和使用方法  

机器学习中的两类参数：通过训练数据学习得到的参数，以及学习算法中需要单独进行优化的参数  
后者称为调优参数，也称为超参，如LR中的正则化系数或者决策树的深度参数 5  
## 6.4.1 使用网格搜索调优超参
> 介绍了网格搜索调优参数的使用方法

网格搜索法的原理是通过我指定的不同超参列表进行暴力穷举搜索，并计算评估每个组合对模型性能的影响  

In [ ]:
'''
python	sklearn	Model Selection	GridSearchCV()
python	sklearn	Model Selection	*.best_score_
'''
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
pipe_svc = Pipeline([('scl', StandardScaler()),
                    ('clf', SVC(random_state=1))])
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
param_grid = [{'clf__C': param_range,
              'clf__kernel': ['linear']},
             {'clf__C': param_range,
              'clf__gamma': param_range,
              'clf__kernel': ['rbf']}]
gs = GridSearchCV(estimator=pipe_svc,
                 param_grid=param_grid,
                 scoring='accuracy',
                 cv=10,
                 n_jobs=-1)
gs = gs.fit(X_train, y_train)
print(gs.best_score_)

In [ ]:
'''
python	sklearn	Model Selection	*.best_params_
'''
print(gs.best_params_)

我们将GridSearchCV的param_grid参数以字典的方式定义为待调优的参数 5  
最后，我们将使用独立的测试数据集，通过GridSearchCV对象的best_estimator_属性对最优模型进行性能评估  

In [ ]:
'''
python	sklearn	Model Selection	*.best_estimator_b
'''
clf = gs.best_estimator_
clf.fit(X_train, y_train)
print('Test accuracy:{:.3f}'.format(clf.score(X_test, y_test)))

虽然网格搜索是寻找最优参数集合的一种强大方法，但评估所有参数组合的计算成本相对高，这时候可以使用随机搜索技术RandomizedSearchCV 5  
## 6.4.2 通过嵌套交叉验证选择算法
> 介绍了嵌套交叉验证的实现原理和使用方法  

在一项对误差估计的偏差情形研究中，Varma和Simon给出了如下结论：使用嵌套交叉验证，估计的真实误差与在测试集上得到的结果几乎没有差距  
下图通过5个外围模块及2个内部模块解释了嵌套交叉验证的概念，这适用于计算性能要求比较高的大规模数据集。这种特殊类型的嵌套交叉验证也称为$5\times2$交叉验证  
![6-5](../syn_pic/py_machine_learning/6-5.png)
我们可以通过如下方式使用嵌套交叉验证 5  

In [ ]:
'''
python	sklearn	Model Selection	GridSearchCV()
python	sklearn	Model Selection	cross_val_score()
'''
gs = GridSearchCV(estimator=pipe_svc, 
                 param_grid=param_grid,
                 scoring='accuracy',
                 cv=10,
                 n_jobs=-1)
scores = cross_val_score(gs, X, y, scoring='accuracy', cv=5)
print('CV accuracy:{:.3f} +/- {:.3f}'.format(np.mean(scores), np.std(scores)))

代码返回的交叉验证准确率平均值对模型超参调优的预期值给出了很好的估计，且使用该值优化过的模型能够预测未知数据  
我们可以使用嵌套交叉验证方法比较SVM模型与简单的决策树分类器  

In [ ]:
'''
python	sklearn	Decision Trees	DecisionTreeClassifier()
'''
from sklearn.tree import DecisionTreeClassifier
gs = GridSearchCV(
        estimator=DecisionTreeClassifier(random_state=0),
        param_grid=[
            {'max_depth': [1, 2, 3, 4, 5, 6, 7, None]}],
        scoring='accuracy',
        cv=5)
scores = cross_val_score(gs,
                        X_train,
                        y_train,
                        scoring='accuracy',
                        cv=5)
print('CV accuracy:{:.3f} +/- {:.3f}'.format(np.mean(scores), np.std(scores)))

嵌套交叉验证对SVM模型性能的评分远高于决策树，由此可判定选择SVM模型 5  
# 6.5 了解不同的性能评价指标
> 介绍了量化评估衡量模型性能的几种常用指标：准确率、召回率、F1分数、ROC曲线及多类别分类的评价 5  

## 6.5.1 读取混淆矩阵
> 介绍了混淆矩阵的概念（理解其他评估指标的基础）和使用方法  

混淆矩阵是展示学习算法性能的一种矩阵，如下图所示：  
![6-6](../syn_pic/py_machine_learning/6-6.png)
sklearn提供了一个方便使用的confusion_matrix函数，使用方法如下  
经测试，实际为左上真负，右上假真，右下真正，左下假负 5  

In [ ]:
'''
python	sklearn	Metrics	confusion_matrix()
'''
from sklearn.metrics import confusion_matrix
pipe_svc.fit(X_train, y_train)
y_pred = pipe_svc.predict(X_test)
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(confmat)

我们可以使用matshow函数将它们表示为上图所示的混淆矩阵形式 

In [ ]:
'''
5
python	matplotlib	axes	matshow()
python	matplotlib	axes	text()
python	matplotlib	cm (colormap)	Colormap reference
python	numpy	The N-dimensional array (ndarray)	shapeb
'''
fig, ax = plt.subplots(figsize=(2.5,2.5))
ax.matshow(confmat, cmap=plt.cm.PuBu, alpha=0.3)
for i in range(confmat.shape[0]):
    for j in range(confmat.shape[1]):
        ax.text(x=j, y=i,
               s=confmat[i, j],
               va='center', ha='center')
plt.xlabel('predicted label')
plt.ylabel('true label')
plt.show()

## 6.5.2 优化分类模型的准确率和召回率
> 介绍了准确率、召回率和F1分数的概念及其使用方法

误差可以理解为预测错误样本的数量与所有被预测样本数量的比值，而准确率计算方法则是正确预测样本的数量与所有被预测样本数量的比值  
预测误差：  
$$ERR=\frac{FP+FN}{FP+FN+TP+TN}$$
预测准确率：  
$$ACC=\frac{TP+TN}{FP+FN+TP+TN}=1-ERR$$
对于类别数量不均衡的分类问题，真正率与假正率是非常有用的性能指标 5  
$$FPR=\frac{FP}{N}=\frac{FP}{FP+TN}$$
$$TPR=\frac{TP}{P}=\frac{TP}{FN+TP}$$
以肿瘤诊断为例，我们更关注正确检测出恶性肿瘤，使得病人得到恰当的治疗  
准确率和召回率是与真正率、真负率相关的性能评价指标，实际上，召回率与真正率含义相同 
$$PRE=\frac{TP}{TP+FP}$$
$$REC=TPR=\frac{TP}{P}=\frac{TP}{FN+TP}$$
5  
在实践中，常采用准确率与召回率的组合，称为F1分数
$$F1=2\frac{PRE\times REC}{PRE+REC}$$
所有这些评分指标均已在sklearn中实现  

In [ ]:
'''
python	sklearn	Metrics	precision_score()
python	sklearn	Metrics	recall_score()
python	sklearn	Metrics	f1_score()
'''
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score, f1_score
print('Precision:{:.3f}'.format(precision_score(y_true=y_test, y_pred=y_pred)))
print('Recall:{:.3f}'.format(recall_score(y_true=y_test, y_pred=y_pred)))
print('F1:{:.3f}'.format(f1_score(y_true=y_test, y_pred=y_pred)))

此外，通过评分参数，我们还可以在网格搜索中使用包括准确率在内的其他多种不同评分标准，详见 https://scikit-learn.org/stable/modules/model_evaluation.html 5  
sklearn中将正类类标表示为1，我们可通过maker_scorer函数来构建我们自己的评分，将正类类标改为0  

In [ ]:
'''
5
python	sklearn	Metrics	make_scorer()
python	sklearn	Model Selection	GridSearchCV()
'''
from sklearn.metrics import make_scorer
scorer = make_scorer(f1_score, pos_label=0)
gs = GridSearchCV(estimator=pipe_svc,
                 param_grid=param_grid,
                 scoring=scorer,
                 cv=10)

## 6.5.3 绘制ROC曲线
> 介绍了ROC曲线基于真正率和假正率衡量性能的实现原理和使用方法  

ROC的对角线可以理解为随机猜测。对于完美的分类器来说，其真正率为1，假正率为0，这时ROC曲线即为横轴0与纵轴1组成的折线  
与ROC曲线类似，我们也可以计算不同概率阈值下一个分类器的准确率-召回率曲线，详见sklearn.metrics.precision_recall_curve  
为了使ROC曲线视觉上更加生动，我们调整了分类器和验证器的设置  

In [ ]:
'''
5
python	sklearn	Model Selection	StratifiedKFold()
python	sklearn	Model Selection	*.split()
python	sklearn	Pipeline	*.fit()
python	sklearn	Pipeline	*predict_proba()
python	sklearn	Metrics	roc_curve()
python	numpy	Mathematical functions	interp()
'''
from sklearn.metrics import roc_curve, auc
X_train2 = X_train[:, [4, 14]]
cv = StratifiedKFold(n_splits=3)
fig = plt.figure(figsize=(7, 5))
mean_tpr = 0.0
mean_fpr = np.linspace(0, 1, 100)
all_tpr = []

for i, (train, test) in enumerate(cv.split(X_train2, y_train)):
    probas = pipe_lr.fit(X_train2[train], y_train[train]).predict_proba(X_train2[test])
    fpr, tpr, thresholds = roc_curve(y_train[test], probas[:, 1], pos_label=1)
    mean_tpr += np.interp(mean_fpr, fpr, tpr)
    mean_tpr[0] = 0.0
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label='ROC fold {} (area = {:.2f})'.format(i+1, roc_auc))

plt.plot([0, 1], [0, 1], linestyle='--', color=(0.6, 0.6, 0.6), label='random guessing')
mean_tpr /= len(list(cv.split(X_train2, y_train)))
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, 'k--', label='mean ROC(area = {:.2f})'.format(mean_auc), lw=2)
plt.plot([0, 0, 1],[0, 1, 1], lw=2, linestyle=':', color='black', label='perfect performance')
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.title('Receiver Operator Characteristic')
plt.legend(loc="lower right")
plt.show()

### 如何理解interp()?
> numpy.interp(x, xp, fp, left=None, right=None, period=None)  
为二维坐标轴的系列点(x,y)提供插值，其中xp是系列点的x坐标ndarray,yp是系列点y坐标的ndarray, x是需要插值点的x坐标ndarray，返回的是需要插值点的y左边ndarray 5  

interp利用三个块数据对ROC曲线的内插均值进行了计算，并使用auc函数计算了低于ROC曲线区域的面积。最终的ROC曲线表明不同的块之间存在一定的方差，平均ROC AUC（0.75）位于最理想情况（1.0）与随机猜测（0.5）之间   
如果我们仅对ROC AUC的得分感兴趣，可以导入roc_auc_score函数  

In [ ]:
'''
python	sklearn	Pipeline	predict()
python	sklearn	Metrics	roc_auc_score()
python	sklearn	Metrics	accuracy_score()
'''
pipe_svc = pipe_svc.fit(X_train2, y_train)
y_pred2 = pipe_svc.predict(X_test[:, [4, 14]])
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
print('ROC AUC:{:.3f}'.format(roc_auc_score(y_true=y_test, y_score=y_pred2)))
print('Accuracy:{:.3f}'.format(accuracy_score(y_true=y_test, y_pred=y_pred2)))

通过ROC AUC得到的分类器性能可以让我们进一步洞悉分类器在类别不均衡样本集合上的性能 5  
### 问题：类别不均衡样本使用ROC AUC和准确率的区别？  
> 待补充

## 6.5.4 多类别分类的评价标准
> 介绍了如何通过sklearn实现多类别分类问题的评价以及其实现机制

sklearn实现了macro宏及micro微均值方法，通过one vs All技术将评分标准扩展到多类别  
k类分类系统的准去率评分的微均值可按如下公式进行计算:  
$$PRE_{micro}=\frac{TP_1+\dots+TP_k}{TP_1+\dots+TP_k+FP_1+\dots+FP_k}$$
宏均值仅计算不同系统的平均分值： 5  
$$PRE_{micro}=\frac{PRE_1+\dots+PRE_k}{k}$$
当我们等同看待每个实例或每次预测时，微均值是有用的，而宏均值则是我们等同看待各个类别，将其用于评估分类器针对最频繁（即样本数量最多的类）类标的整体性能  
当我们衡量sklearn中多类别分类模型，会默认使用一个归一化项或是宏均值的一个加权变种  
当数据中类中样本分布不均衡时，也就是类标数量不一致时，采用加权宏均值比较有效  
我们可以通过sklearn.metrics模块导入其他不同的评分函数，如precision_score或make_scorer函数等，并利用函数内置的average参数定义平均方法 5  

In [ ]:
'''
5
python	sklearn	Model selection and evaluation	Multiclass and multilabel classification
python	sklearn	Metrics	precision_score()
python	sklearn	Metrics	make_scorer()
'''
pre_scorer = make_scorer(score_func=precision_score, 
                        pos_label=1,
                        greater_is_better=True,
                        average='micro')

# 本章小结
我们介绍了通过便捷的流水线模型串联不同的数据转换技术与分类器，以帮助我们高效地训练与评估机器学习模型  
我们使用流水线进行k折交叉验证，k折交叉验证是模型选择及评估的一种基本技术  
使用k折交叉验证，我们绘制了学习曲线和验证曲线以诊断学习算法中过拟合与欠拟合等常见问题  
使用网格搜索进一步对模型进行微调  
最后我们学习了混淆矩阵以及各种不同的性能评价指标，主要应用在针对特定问题需要进一步优化模型时 5  